<a href="https://colab.research.google.com/github/renxingwill/GPT-SoVITS/blob/main/GPT-SoVITS-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-SoVITS Colab Inference
此笔记本用于在 Google Colab 上运行 GPT-SoVITS 推理 WebUI。

**使用步骤：**
1. 运行 **环境配置** (只需运行一次)。
2. 运行 **下载模型** (选择你需要的模型)。
3. 运行 **启动 WebUI**。

## 1. 环境配置 (Env Setup)
安装依赖库和 Conda 环境。此步骤耗时较长（约 3-5 分钟），请耐心等待。

In [ ]:
#@title 安装依赖 (Install Dependencies)
import os

# 1. 创建 setup.sh 脚本
setup_script = """
set -e
cd /content

if [ ! -d "GPT-SoVITS" ]; then
    git clone https://github.com/RVC-Boss/GPT-SoVITS.git
fi

cd GPT-SoVITS
mkdir -p GPT_weights
mkdir -p SoVITS_weights

# 检查并创建 Conda 环境
if conda env list | awk '{print $1}' | grep -Fxq "GPTSoVITS"; then
    echo "Environment GPTSoVITS already exists."
else
    conda create -n GPTSoVITS python=3.10 -y
fi

source activate GPTSoVITS
pip install ipykernel

# 安装依赖
bash install.sh --device CU126 --source HF
"""

with open("/content/setup.sh", "w") as f:
    f.write(setup_script)

# 2. 安装 CondaColab 并运行脚本
try:
    import condacolab
    if not os.path.exists("/usr/local/conda-meta/pinned"):
        condacolab.install_from_url("https://repo.anaconda.com/archive/Anaconda3-2024.10-1-Linux-x86_64.sh")
except ImportError:
    !pip install -q condacolab
    import condacolab
    condacolab.install_from_url("https://repo.anaconda.com/archive/Anaconda3-2024.10-1-Linux-x86_64.sh")

print("正在配置环境，请稍候...")
!cd /content && bash setup.sh
print("环境配置完成！")

## 2. 下载模型 (Download Models)
这里整合了 HuggingFace 和 ModelScope 的下载选项。你可以修改下面的参数来下载不同的模型。

In [ ]:
#@title 选择下载源和模型 (Select Model Source)
import os

Model_Source = "HuggingFace" #@param ["HuggingFace", "ModelScope"]

#@markdown ### HuggingFace 设置
HF_User_ID = "AkitoP" #@param {type:"string"}
HF_Repo_Name = "GPT-SoVITS-v2-aegi" #@param {type:"string"}
HF_Branch = "main" #@param {type:"string"}
HF_GPT_Path = "new_aegigoe-e100.ckpt" #@param {type:"string"}
HF_SoVITS_Path = "new_aegigoe_e60_s32220.pth" #@param {type:"string"}

#@markdown ### ModelScope 设置
MS_User_ID = "aihobbyist" #@param {type:"string"}
MS_Repo_Name = "GPT-SoVits-V2-models" #@param {type:"string"}
MS_Branch = "master" #@param {type:"string"}
MS_GPT_Path = "Genshin_Impact/EN/GPT_GenshinImpact_EN_5.1.ckpt" #@param {type:"string"}
MS_SoVITS_Path = "Wuthering_Waves/CN/SV_WutheringWaves_CN_1.3.pth" #@param {type:"string"}

#@markdown ### 预训练底模 (推荐下载)
#@markdown 如果你没有底模，建议勾选此项，否则可能报错。
Download_Pretrained_Models = True #@param {type:"boolean"}

def download_file(url, save_path):
    if not os.path.exists(save_path):
        print(f"Downloading {url} to {save_path}...")
        !wget -O "{save_path}" "{url}"
    else:
        print(f"File {save_path} already exists. Skipping.")

if Download_Pretrained_Models:
    print("Downloading Pretrained Base Models...")
    !cd /content/GPT-SoVITS/GPT_weights && wget -nc https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt
    !cd /content/GPT-SoVITS/SoVITS_weights && wget -nc https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s2G488k.pth

if Model_Source == "HuggingFace":
    HF_BASE = "https://huggingface.co"
    REPO_ID = f"{HF_User_ID}/{HF_Repo_Name}"
    GPT_URL = f"{HF_BASE}/{REPO_ID}/resolve/{HF_Branch}/{HF_GPT_Path}"
    SOVITS_URL = f"{HF_BASE}/{REPO_ID}/resolve/{HF_Branch}/{HF_SoVITS_Path}"
    
    download_file(GPT_URL, f"/content/GPT-SoVITS/GPT_weights/{os.path.basename(HF_GPT_Path)}")
    download_file(SOVITS_URL, f"/content/GPT-SoVITS/SoVITS_weights/{os.path.basename(HF_SoVITS_Path)}")

elif Model_Source == "ModelScope":
    MS_BASE = "https://www.modelscope.cn/models"
    REPO_ID = f"{MS_User_ID}/{MS_Repo_Name}"
    GPT_URL = f"{MS_BASE}/{REPO_ID}/resolve/{MS_Branch}/{MS_GPT_Path}"
    SOVITS_URL = f"{MS_BASE}/{REPO_ID}/resolve/{MS_Branch}/{MS_SoVITS_Path}"

    download_file(GPT_URL, f"/content/GPT-SoVITS/GPT_weights/{os.path.basename(MS_GPT_Path)}")
    download_file(SOVITS_URL, f"/content/GPT-SoVITS/SoVITS_weights/{os.path.basename(MS_SoVITS_Path)}")

print("模型下载完成！")

## 3. 启动 WebUI (Launch)
运行下方代码后，请点击输出栏底部的 `Running on public URL: https://xxxx.gradio.live` 链接。

In [ ]:
#@title 启动 WebUI
!cd /content/GPT-SoVITS && source activate GPTSoVITS && export is_share=True && python webui.py